In [26]:
import mysql.connector as sc
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix, RocCurveDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.svm import SVR

import joblib
import re
import os


In [27]:
import mysql.connector as sc

conn = sc.connect(
    host='localhost',
    user='root',
    password='Root@123',
    database='pradip',
    port=3306
)

cursor = conn.cursor()

sql = """
SELECT hist_recency_days,
       hist_total_orders,
       hist_total_revenue,
       mh_clicks,
       mh_opens,
       mh_click_rate,
       th_total_tickets,
       th_resolved_tickets,
       th_closed_tickets,
       loyalty_tier,
       churn_label
FROM vw_churn_training;
"""

cursor.execute(sql)
data = cursor.fetchall()

print(data)
print(len(data))


[(None, 0, 0.0, Decimal('1'), Decimal('0'), Decimal('1.0000'), 1, Decimal('0'), Decimal('0'), 'Silver', 1), (None, 0, 0.0, Decimal('0'), Decimal('0'), Decimal('0.0000'), 1, Decimal('1'), Decimal('0'), 'Platinum', 1), (None, 0, 0.0, Decimal('0'), Decimal('0'), Decimal('0.0000'), 3, Decimal('1'), Decimal('1'), 'Silver', 1), (None, 0, 0.0, Decimal('0'), Decimal('3'), Decimal('0.0000'), 0, Decimal('0'), Decimal('0'), 'Gold', 1), (None, 0, 0.0, Decimal('0'), Decimal('0'), Decimal('0.0000'), 2, Decimal('0'), Decimal('0'), 'Platinum', 1), (None, 0, 0.0, Decimal('1'), Decimal('1'), Decimal('0.5000'), 0, Decimal('0'), Decimal('0'), 'Silver', 1), (None, 0, 0.0, Decimal('2'), Decimal('2'), Decimal('0.5000'), 1, Decimal('1'), Decimal('0'), 'Silver', 1), (None, 0, 0.0, Decimal('0'), Decimal('1'), Decimal('0.0000'), 0, Decimal('0'), Decimal('0'), 'Bronze', 1), (None, 0, 0.0, Decimal('0'), Decimal('0'), Decimal('0.0000'), 2, Decimal('1'), Decimal('0'), 'Silver', 1), (None, 0, 0.0, Decimal('0'), Decim

In [28]:
df = pd.DataFrame(data, columns=[
    "hist_recency_days",
    "hist_total_orders",
    "hist_total_revenue",
    "mh_clicks",
    "mh_opens",
    "mh_click_rate",
    "th_total_tickets",
    "th_resolved_tickets",
    "th_closed_tickets",
    "loyalty_tier",
    "churn_label"
])


In [29]:
df.head(5)

,hist_recency_days,hist_total_orders,hist_total_revenue,mh_clicks,mh_opens,mh_click_rate,th_total_tickets,th_resolved_tickets,th_closed_tickets,loyalty_tier,churn_label
0,NaN,0,0.0,1,0,1.0000,1,0,0,Silver,1
1,NaN,0,0.0,0,0,0.0000,1,1,0,Platinum,1
2,NaN,0,0.0,0,0,0.0000,3,1,1,Silver,1
3,NaN,0,0.0,0,3,0.0000,0,0,0,Gold,1
4,NaN,0,0.0,0,0,0.0000,2,0,0,Platinum,1


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   hist_recency_days    9149 non-null   float64
 1   hist_total_orders    12000 non-null  int64  
 2   hist_total_revenue   12000 non-null  float64
 3   mh_clicks            12000 non-null  object 
 4   mh_opens             12000 non-null  object 
 5   mh_click_rate        11265 non-null  object 
 6   th_total_tickets     12000 non-null  int64  
 7   th_resolved_tickets  12000 non-null  object 
 8   th_closed_tickets    12000 non-null  object 
 9   loyalty_tier         12000 non-null  object 
 10  churn_label          12000 non-null  int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 1.0+ MB


In [31]:
df.describe()

,hist_recency_days,hist_total_orders,hist_total_revenue,th_total_tickets,churn_label
count,9149.000000,12000.000000,12000.000000,12000.000000,12000.000000
mean,287.343316,2.063583,49766.289383,1.131667,0.872000
std,227.906272,1.710736,49950.250497,1.024167,0.334104
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,100.000000,1.000000,2890.562439,0.000000,1.000000
50%,227.000000,2.000000,38122.889648,1.000000,1.000000
75%,432.000000,3.000000,78952.917633,2.000000,1.000000
max,912.000000,8.000000,299071.572266,6.000000,1.000000


In [32]:
df.mh_click_rate.unique()

array([Decimal('1.0000'), Decimal('0.0000'), Decimal('0.5000'), None,
       Decimal('0.2500'), Decimal('0.3333'), Decimal('0.6667'),
       Decimal('0.2000'), Decimal('0.6000'), Decimal('0.7500'),
       Decimal('0.4000'), Decimal('0.1667'), Decimal('0.1429'),
       Decimal('0.1250'), Decimal('0.4286'), Decimal('0.2857'),
       Decimal('0.3750'), Decimal('0.6250'), Decimal('0.8000'),
       Decimal('0.8333'), Decimal('0.7143'), Decimal('0.5714'),
       Decimal('0.8571')], dtype=object)

In [33]:
df["mh_click_rate"] = df["mh_click_rate"].fillna(df["mh_click_rate"].median())


In [34]:
df["hist_recency_days"] = df["hist_recency_days"].fillna(df["hist_recency_days"].median())


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   hist_recency_days    12000 non-null  float64
 1   hist_total_orders    12000 non-null  int64  
 2   hist_total_revenue   12000 non-null  float64
 3   mh_clicks            12000 non-null  object 
 4   mh_opens             12000 non-null  object 
 5   mh_click_rate        12000 non-null  object 
 6   th_total_tickets     12000 non-null  int64  
 7   th_resolved_tickets  12000 non-null  object 
 8   th_closed_tickets    12000 non-null  object 
 9   loyalty_tier         12000 non-null  object 
 10  churn_label          12000 non-null  int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 1.0+ MB


In [36]:
print(df["hist_recency_days"])
df.head(10)

0        227.0
1        227.0
2        227.0
3        227.0
4        227.0
         ...  
11995     19.0
11996    227.0
11997     85.0
11998    509.0
11999    263.0
Name: hist_recency_days, Length: 12000, dtype: float64


,hist_recency_days,hist_total_orders,hist_total_revenue,mh_clicks,mh_opens,mh_click_rate,th_total_tickets,th_resolved_tickets,th_closed_tickets,loyalty_tier,churn_label
0,227.0,0,0.0,1,0,1.0000,1,0,0,Silver,1
1,227.0,0,0.0,0,0,0.0000,1,1,0,Platinum,1
2,227.0,0,0.0,0,0,0.0000,3,1,1,Silver,1
3,227.0,0,0.0,0,3,0.0000,0,0,0,Gold,1
4,227.0,0,0.0,0,0,0.0000,2,0,0,Platinum,1
5,227.0,0,0.0,1,1,0.5000,0,0,0,Silver,1
6,227.0,0,0.0,2,2,0.5000,1,1,0,Silver,1
7,227.0,0,0.0,0,1,0.0000,0,0,0,Bronze,1
8,227.0,0,0.0,0,0,0.0000,2,1,0,Silver,1
9,227.0,0,0.0,0,0,0.0000,1,1,0,Gold,1


In [37]:
df.describe()

,hist_recency_days,hist_total_orders,hist_total_revenue,th_total_tickets,churn_label
count,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000
mean,273.006750,2.063583,49766.289383,1.131667,0.872000
std,200.647687,1.710736,49950.250497,1.024167,0.334104
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,137.000000,1.000000,2890.562439,0.000000,1.000000
50%,227.000000,2.000000,38122.889648,1.000000,1.000000
75%,356.000000,3.000000,78952.917633,2.000000,1.000000
max,912.000000,8.000000,299071.572266,6.000000,1.000000


In [38]:
print(list(df.columns))

['hist_recency_days', 'hist_total_orders', 'hist_total_revenue', 'mh_clicks', 'mh_opens', 'mh_click_rate', 'th_total_tickets', 'th_resolved_tickets', 'th_closed_tickets', 'loyalty_tier', 'churn_label']


In [39]:
print(df['loyalty_tier'].unique())

['Silver' 'Platinum' 'Gold' 'Bronze']


In [40]:
# from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["loyalty_tier"] = le.fit_transform(df["loyalty_tier"])


In [41]:
print(df.head())

   hist_recency_days  hist_total_orders  hist_total_revenue mh_clicks  \
0              227.0                  0                 0.0         1   
1              227.0                  0                 0.0         0   
2              227.0                  0                 0.0         0   
3              227.0                  0                 0.0         0   
4              227.0                  0                 0.0         0   

  mh_opens mh_click_rate  th_total_tickets th_resolved_tickets  \
0        0        1.0000                 1                   0   
1        0        0.0000                 1                   1   
2        0        0.0000                 3                   1   
3        3        0.0000                 0                   0   
4        0        0.0000                 2                   0   

  th_closed_tickets  loyalty_tier  churn_label  
0                 0             3            1  
1                 0             2            1  
2                

In [42]:
print(df['loyalty_tier'].unique())

[3 2 1 0]


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   hist_recency_days    12000 non-null  float64
 1   hist_total_orders    12000 non-null  int64  
 2   hist_total_revenue   12000 non-null  float64
 3   mh_clicks            12000 non-null  object 
 4   mh_opens             12000 non-null  object 
 5   mh_click_rate        12000 non-null  object 
 6   th_total_tickets     12000 non-null  int64  
 7   th_resolved_tickets  12000 non-null  object 
 8   th_closed_tickets    12000 non-null  object 
 9   loyalty_tier         12000 non-null  int64  
 10  churn_label          12000 non-null  int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 1.0+ MB


In [44]:
df['mh_clicks'] = df['mh_clicks'].astype(int)
df['mh_opens'] = df['mh_opens'].astype(int)
df['mh_click_rate'] = df['mh_click_rate'].astype(float)
df['th_resolved_tickets'] = df['th_resolved_tickets'].astype(int)
df['th_closed_tickets'] = df['th_closed_tickets'].astype(int)


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   hist_recency_days    12000 non-null  float64
 1   hist_total_orders    12000 non-null  int64  
 2   hist_total_revenue   12000 non-null  float64
 3   mh_clicks            12000 non-null  int64  
 4   mh_opens             12000 non-null  int64  
 5   mh_click_rate        12000 non-null  float64
 6   th_total_tickets     12000 non-null  int64  
 7   th_resolved_tickets  12000 non-null  int64  
 8   th_closed_tickets    12000 non-null  int64  
 9   loyalty_tier         12000 non-null  int64  
 10  churn_label          12000 non-null  int64  
dtypes: float64(3), int64(8)
memory usage: 1.0 MB


In [47]:
x=df[['hist_recency_days','hist_total_orders','hist_total_revenue','mh_clicks','mh_opens','mh_click_rate','th_total_tickets','th_resolved_tickets','th_closed_tickets','loyalty_tier']]
y=df['churn_label']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [48]:
log_reg=LogisticRegression(max_iter=1000)
log_reg.fit(x_train,y_train)
pred_l=log_reg.predict(x_test)
proba_lr=log_reg.predict_proba(x_test)[:,1]

print("=== Logistic Regression ===")
print('Accuracy',accuracy_score(y_test,pred_l))


=== Logistic Regression ===
Accuracy 0.8745833333333334


In [50]:
with open("vw_churn_training_lr.pkl","wb") as f:
    pickle.dump(log_reg,f)